# Bayesian Neural Networks to Predict Hard Landing with DASHlink Data
Authors: Dr. Yingxiao Kong, Vanderbilt University

Email: yingxiao.kong@vanderbit.edu

## Overview of Research
In this work, we use an open-source dataset - [NASA's DASHlink data](https://c3.ndc.nasa.gov/dashlink/) - to isolate data for landing aircraft that both have hard landing and normal landing occurrences. The objective is to use [this sample data](https://c3.ndc.nasa.gov/dashlink/projects/85/resources/?type=ds) to train a Bayesian Neural Network model to predict touchdown vertical speed for a landing aircraft with the intent to use as a screening for identifying hard landing events before they occur.

This series of Jupyter notebook demonstrations into 3 modules. The presented module is in **bold**:
- Module 1 - Download DASHlink Data
- Module 2 - DASHlink Data Pre-Processing and Feature Selection with Maximum Relevance and Minimum Reduandancy (MRMR)
- **Module 3 - Bayesian Neural Network Model Training**

## Module 3: Bayesian Neural Network Model Training

## Step 3a: Get Processed Data and Ordered Features from Module 2

In [64]:
import pandas as pd
df_landing =  pd.read_csv('processed_data_landing_at_msp.csv')
df_features=pd.read_csv('ordered_features.csv')

In [65]:
df_landing

,LATP,LONP,MSQT_1,BAL1,TAS,GS,TH,FLAP,GLS,LOC,...,CWPC,WS,WD,ALTR,TD_ALTR,TD_LAT,TD_LON,TD_ALT,heights,DIST
0,44.891682,-93.241822,1.0,1006.643045,117.933373,115.007588,125.581422,3645.0,0.014040,-0.003136,...,1782.000000,10.948235,-159.834434,-536.630359,-104.780302,44.886191,-93.228955,806.643045,200,1185.595409
1,44.888764,-93.234960,1.0,906.643045,115.530863,111.638458,125.322379,3645.0,0.183690,0.003136,...,1795.861409,9.932823,-165.738392,-709.834437,-104.780302,44.886191,-93.228955,806.643045,100,553.902372
2,44.887734,-93.232386,1.0,856.643045,113.021779,110.749442,123.876531,3645.0,0.133380,0.003332,...,1672.000000,6.212059,-163.252476,-823.996865,-104.780302,44.886191,-93.228955,806.643045,50,320.767080
3,44.887502,-93.231929,1.0,846.643045,112.537305,110.196048,124.172654,3645.0,0.136028,0.002001,...,1718.580098,6.975467,-166.607074,-688.934215,-104.780302,44.886191,-93.228955,806.643045,40,276.478561
4,44.887351,-93.231586,1.0,836.643045,111.188473,109.765476,124.137817,3645.0,0.133199,0.002040,...,1748.036175,6.994513,-160.733711,-773.697501,-104.780302,44.886191,-93.228955,806.643045,30,244.578379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,44.882928,-93.195317,1.0,786.438823,113.839470,122.158225,-59.134194,3645.0,0.069030,0.001176,...,1467.000000,6.996122,128.635156,-400.249286,-271.354242,44.883445,-93.196665,778.438823,8,121.006367
3872,44.882928,-93.195317,1.0,784.438823,113.909336,121.361765,-58.945564,3645.0,0.069030,0.001176,...,1581.000000,6.408334,123.705862,-285.024058,-271.354242,44.883445,-93.196665,778.438823,6,121.006367
3873,44.883100,-93.195807,1.0,782.438823,113.571076,120.748203,-58.783896,3645.0,0.307710,0.001568,...,1665.000000,5.906483,119.784757,-376.884980,-271.354242,44.883445,-93.196665,778.438823,4,77.845605
3874,44.883100,-93.195807,1.0,780.438823,111.655264,119.874082,-58.497770,3645.0,0.307710,0.001568,...,1296.000000,5.595276,111.880163,-259.560171,-271.354242,44.883445,-93.196665,778.438823,2,77.845605


In [66]:
df_features

,0.0,0.25,0.5,0.75,1.0
0,ALTR,ALTR,ALTR,ALTR,ALTR
1,CCPC,CCPC,CCPC,CCPC,CCPC
2,PTCH,PTCH,BLAC,BLAC,BLAC
3,ELEV_1,ELEV_1,PTCH,PTCH,GLS
4,N1_1,N1_1,ELEV_1,ELEV_1,PTCH
5,GS,GS,N1_1,N1_1,ELEV_1
6,TAS,TAS,GS,GS,N1_1
7,WS,WS,TAS,TAS,GS
8,DIST,DIST,WS,WS,TAS
9,BAL1,BAL1,DIST,DIST,WS


### Step 3b: Define Training Inputs and Output Function

In [67]:
def get_train_data(df,output='TD_ALTR',drop='heights'):
    if 'heights' in df.columns:
        df.drop(labels='heights',axis=1,inplace=True)
    
    df_in = df.drop(labels=output,axis=1)
    df_out = df[output]
    x_train = (df_in-df_in.min())/(df_in.max()-df_in.min())

    y_train= df_out.values*0.3048/60

    y_train_scale = np.log(y_train - min(y_train) + 1)
    ### here we also scale down RNN_y_scale to make it eaiser to converge
    delta=np.log(-2-min(y_train)+1)
    
    y_train_mean = np.mean(y_train)
    y_train_std = np.std(y_train)
    z_train = (y_train-y_train_mean)/(y_train_std)

    y_train_scale_reverse = np.exp(y_train_scale)-1+np.min(y_train)

    diff = np.mean(y_train)-np.mean(y_train_scale_reverse)
                                                                                
    return x_train,y_train,y_train_scale,delta,z_train,diff

### Step 3c: Define RNN Models

In [68]:
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Dropout,LSTM,Activation
from keras.optimizers import Adam
from keras.regularizers import l1,l2
from keras.models import Sequential

def get_RNN_model(in_shape, idrop=0.25, odrop=0.25, rdrop=0.25, weight_decay=1e-4, lr=1e-3,num_unit=100):

    model=Sequential()
    model.add(LSTM(num_unit,kernel_regularizer=l2(weight_decay),recurrent_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay),dropout=idrop,recurrent_dropout=rdrop,input_shape=(None,in_shape),\
                  kernel_initializer='random_uniform',return_sequences=True))

    model.add(Activation('relu'))
    
    model.add(LSTM(num_unit,dropout=idrop,recurrent_dropout=rdrop,return_sequences=False,kernel_regularizer=l2(weight_decay),recurrent_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay),kernel_initializer='random_uniform',))
    model.add(Activation('relu'))
    if odrop:
        model.add(Dropout(odrop))
    model.add(Dense(1,activation='linear',kernel_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay)))
    optimizer_=Adam(lr)
    #in the paper variational dropout, learning rate isn't considered
#     optimizer=Adam
    model.compile(loss='mse',metrics=['mse'],optimizer=optimizer_)
    return model

def get_RNN_model_2(in_shape, idrop=0.25, odrop=0.25, rdrop=0.25, weight_decay=1e-4, lr=1e-3,num_unit=100):

    model=Sequential()
    model.add(LSTM(num_unit,kernel_regularizer=l1(weight_decay),recurrent_regularizer=l1(weight_decay),bias_regularizer=l1(weight_decay),dropout=idrop,recurrent_dropout=rdrop,input_shape=(None,in_shape),\
                  kernel_initializer='random_uniform',return_sequences=True))

    model.add(Activation('relu'))
    
    model.add(LSTM(num_unit,dropout=idrop,recurrent_dropout=rdrop,return_sequences=False,kernel_regularizer=l1(weight_decay),recurrent_regularizer=l1(weight_decay),bias_regularizer=l1(weight_decay)))
    model.add(Activation('relu'))
    if odrop:
        model.add(Dropout(odrop))
    model.add(Dense(1,activation='linear',kernel_regularizer=l1(weight_decay),bias_regularizer=l1(weight_decay)))
    optimizer_=Adam(lr)
    #in the paper variational dropout, learning rate isn't considered
#     optimizer=Adam
    model.compile(loss='mse',metrics=['mse'],optimizer=optimizer_)
    return model
    
    

class KerasDPprediction(object):
    def __init__(self,model):
        self.f= K.function([model.layers[0].input,K.learning_phase()],[model.layers[-1].output])
        
    def predict(self,x,n_iter=1000):
        result=[]
        for _ in range(n_iter):
            result.append(np.squeeze(self.f([x,1])))
        result = np.array(result)
        
        return result

### Step 3c: Train RNNs for Given MRMR Result

In [69]:
MIN_REDUNDANCY_WEIGHT = 0.5
OUTPUT = 'TD_ALTR'
input_order=df_features.iloc[:,df_features.columns=='{}'.format(MIN_REDUNDANCY_WEIGHT)]
print(input_order)

       0.5
0     ALTR
1     CCPC
2     BLAC
3     PTCH
4   ELEV_1
5     N1_1
6       GS
7      TAS
8       WS
9     DIST
10    BAL1
11    FPAC
12    LONP
13    LATP
14      TH
15     GLS
16    ROLL
17    FLAP
18    CTAC
19     TRK
20    RUDD
21     LOC
22      WD
23   AIL_1


In [70]:
import numpy as np
NINPUTS = np.arange(4,24,4)
print(NINPUTS)

[ 4  8 12 16 20]


In [71]:
grpby = df_landing.groupby(by='heights')
NINPUTS = np.arange(4,24,4)
for i,g in enumerate(grpby):
    height = g[0]
    df = g[1]
    for n in NINPUTS:
        i = list(input_order.values[:n,0].reshape(1,n)[0])
        df_sub=df[i+[OUTPUT]]
        x_train,y_train,y_train_scale,delta,z_train,diff = get_train_data(df_sub)
          
        ## TRAIN RNN Codes - OLD CODE BELOW###

In [ ]:
# for j in range(1):
#     print('j: '+str(j))
#     sort_index = []
#     sele_col = 2
#     ### sele_col: 0-4
#     for i in range(all_para.shape[0]):
#         if all_para[i,sele_col] in sele_key_list:
#             sort_index.append(sele_key_list.index(all_para[i,sele_col]))
# #     sort_index = np.insert(sort_index,0,23)

#     final_RNN_x = RNN_x[:,:,sort_index]
#     all_result=[]
#     all_RMSE=[]
#     all_MAE = []
#     for i in np.arange(4,28,4):
#         print('i: '+str(i))
#         ### i is the number of parameters taken to construct the training model
#         partial_data = train_data(final_RNN_x,i)
#         #### the model is trained based on RNN_y_scale_reverse
#         RNN_x_train,RNN_x_test,RNN_y_train,RNN_y_test=train_test_split(partial_data[:,:,:], RNN_y_scale_reverse, test_size=0.2, random_state=40)
#     #     RNN_x_train,RNN_x_test,RNN_y_train,RNN_y_test=train_test_split(partial_data[:,:,:],RNN_y, test_size=0.2, random_state=40)
#         RNN_model = get_RNN_model(RNN_x_train.shape[2])
#         RNN_model.fit(RNN_x_train,RNN_y_train,batch_size=30,verbose=False,epochs=200)
#         kdp = KerasDPprediction(RNN_model)
#         y_test_pred=kdp.predict(RNN_x_test,1000)
#         all_result.append(y_test_pred)
#         mean_y_test_pred = np.mean(y_test_pred,axis=0)
#         RMSE =np.sqrt(np.mean((mean_y_test_pred-RNN_y_test)**2)) 
#         MAE = np.mean(np.abs(mean_y_test_pred-RNN_y_test))

#         all_RMSE.append(RMSE)
#         all_MAE.append(MAE)
#     all_all_result.append(all_result)
#     all_all_RMSE.append(all_RMSE)
#     all_all_MAE.append(all_MAE

### Step 3d: Assessment of Model Results

In [ ]:
import matplotlib.pyplot as plt
plt.plot(all_all_MAE[0])